In [1]:
import pandas as pd
import xarray as xr

from ALLCools.mcds.MCDS import MCDS

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/typing.py:842: FutureWarning: xarray subclass MCDS should explicitly define __slots__
  super().__init_subclass__(*args, **kwargs)


In [2]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
               )

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cell_tidy_data = cell_tidy_data[cell_tidy_data['MajorType'] == 'PT-L5']
cell_tidy_data.to_msgpack('CellTidyData.PT-L5.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  


In [4]:
mcds_pattern = '/home/hanliu/project/mouse_rostral_brain/dataset/*.mcds'

In [6]:
import pathlib
mcds_paths = [
    str(i) for i in list(
        pathlib.Path('/home/hanliu/project/mouse_rostral_brain/dataset/').glob(
            '*.mcds'))[:-3]
]


def read_netcdfs(paths, dim):
    # glob expands paths with * to a list of files, like the unix shell
    datasets = []
    for p in paths:
        print(p)
        ds = xr.open_dataset(p)
        cells = ds.get_index('cell')
        cells = cells[cells.isin(cell_tidy_data.index)]
        if cells.size == 0:
            continue
        else:
            datasets.append(ds.sel(cell=cells).load())
    combined = xr.concat(datasets, dim)
    return combined

mcds = read_netcdfs(mcds_paths, dim='cell')

/home/hanliu/project/mouse_rostral_brain/dataset/4H-180806.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/11E-190214.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5A-190205.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/11E-190305.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/4H-180911.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/11F-190214.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5A-190307.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/11F-190305.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5B-180514.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/1A-180226.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5B-180529.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/1A-180227.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5F-181218.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/1B-180119.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/5F-181220.mcds
/home/hanliu/project/mouse_rostral_b

In [7]:
mcds

<xarray.Dataset>
Dimensions:              (cell: 2638, chrom100k: 27269, count_type: 2, gene: 55487, mc_type: 2, strand_type: 1)
Coordinates:
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chrY' 'chrM'
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 91744698 16299
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 91700000 0
    gene_start           (gene) int64 3073252 3102015 3205900 ... 15288 15355
    gene_end             (gene) int64 3074321 3102124 3671497 ... 15354 15421
    gene_chrom           (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 27265 27266 27267 27268
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * cell                 (cell) object '11E_M_100' '11E_M_1184' ... '5D_M_2370'
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 2 ... 0
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 0 ... 1401

In [8]:
mcds.to_netcdf('PT-L5.mcds')

In [9]:
!pwd

/home/hanliu/project/mouse_rostral_brain/study/L5-PT_Integrate_with_RS2
